In [34]:
import os
import sys
import subprocess
import numpy as np
import itk
# Add the 'src' folder to the Python path
sys.path.append(os.path.abspath('../'))
from src.DELAT_utils import match_h5_files_by_channels, setup_logging, collect_region_stats_paths
from src.registration import register_and_transform
from src.stats import compute_region_stats
import logging
from tifffile import imwrite

# single animal

In [20]:
animal = 'ANM543607_BM19'
base_dir = '/nrs/spruston/Boaz/I2/2024-07-31_Mafe/'

In [2]:
animal = 'ANM550749_left_JF552'
base_dir = '/nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains'

In [21]:
# Set the parameters manually here

fx_path = '/nrs/spruston/Boaz/I2/atlas10_hemi.tif'
param_files_dir = '/nrs/spruston/Boaz/I2/itk'
annotation_path = '/nrs/spruston/Boaz/I2/annotation_10_hemi.nii'

In [22]:
logger = setup_logging(base_dir, animal)
# Load the fixed image (fx)
logger.info(f"Loading fixed image (fx) from {fx_path}.")
fx = itk.imread(fx_path, pixel_type=itk.US)

# Load the parameter files
param_files = [
    os.path.join(param_files_dir, 'Order1_Par0000affine.txt'),
    os.path.join(param_files_dir, 'Order3_Par0000bspline.txt'),
    os.path.join(param_files_dir, 'Order4_Par0000bspline.txt'),
    os.path.join(param_files_dir, 'Order5_Par0000bspline.txt')
]
logger.info(f"Loaded parameter files from {param_files_dir}.")



2024-11-14 14:29:31 - INFO - Logging initialized for animal processing.
2024-11-14 14:29:31 - INFO - Loading fixed image (fx) from /nrs/spruston/Boaz/I2/atlas10_hemi.tif.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
2024-11-14 14:29:34 - INFO - Loaded parameter files from /nrs/spruston/Boaz/I2/itk.


Original image:
 [[[ 70000      2      3]
  [     4      5  65536]]

 [[ 65537 100000      7]
  [     8      9     10]]]
Converted image:
 [[[ 6  2  3]
  [ 4  5  0]]

 [[ 1 11  7]
  [ 8  9 10]]]


In [23]:
# Load the annotation volume
logger.info(f"Loading annotation volume from {annotation_path}.")
itk_annotation = itk.imread(annotation_path, itk.ULL)
annotation_np = itk.array_view_from_image(itk_annotation)
annotation_np.dtype, annotation_np.shape


2024-11-14 14:29:34 - INFO - Loading annotation volume from /nrs/spruston/Boaz/I2/annotation_10_hemi.nii.


(dtype('uint64'), (800, 1320, 658))

In [25]:
import numpy as np

def convert_uint64_to_uint16(image):
    # Maximum value for uint16
    max_uint16 = np.iinfo(np.uint16).max

    # Find unique values in the image
    unique_values = np.unique(image)

    # Identify values greater than max_uint16
    overflow_values = unique_values[unique_values > max_uint16]

    # Find all available uint16 values
    available_uint16_values = np.setdiff1d(np.arange(max_uint16 + 1), unique_values[unique_values <= max_uint16])

    # Create a dictionary to map overflow values to available uint16 values
    if len(overflow_values) > len(available_uint16_values):
        raise ValueError("Not enough available uint16 values to remap all overflow values.")

    overflow_mapping = dict(zip(overflow_values, available_uint16_values[:len(overflow_values)]))

    # Apply the mapping to convert the image
    converted_image = image.copy()
    for overflow_value, new_value in overflow_mapping.items():
        converted_image[converted_image == overflow_value] = new_value

    # Convert to uint16
    converted_image = converted_image.astype(np.uint16)
    
    return converted_image

annotation_np_uint16 = convert_uint64_to_uint16(annotation_np)


In [32]:
import numpy as np

def remap_uint16_scrambled_contrast(image):
    # Ensure the image is uint16
    if image.dtype != np.uint16:
        raise ValueError("Input image must be of type uint16.")
    
    # Find unique values in the image
    unique_values = np.unique(image)
    num_unique = len(unique_values)
    
    # Maximum value for uint16
    max_uint16 = np.iinfo(np.uint16).max

    # Generate linearly spaced values from 1 to max_uint16
    linear_values = np.linspace(1, max_uint16, num=num_unique, dtype=np.uint16)
    
    # Shuffle the linear values to scramble the mapping
    np.random.shuffle(linear_values)
    
    # Create a mapping dictionary with scrambled values
    scrambled_mapping = dict(zip(unique_values, linear_values))
    
    # Apply the scrambled mapping to the image
    remapped_image = np.vectorize(scrambled_mapping.get)(image)
    
    # Ensure output is uint16
    remapped_image = remapped_image.astype(np.uint16)
    
    return remapped_image

# Test the function with a sample uint16 image
test_image = np.array([
    [[100, 2, 3], [4, 5, 65535]],
    [[30000, 50000, 7], [8, 9, 10]]
], dtype=np.uint16)

remapped_image = remap_uint16_scrambled_contrast(test_image)
print("Original image:\n", test_image)
print("Remapped image:\n", remapped_image)



Original image:
 [[[  100     2     3]
  [    4     5 65535]]

 [[30000 50000     7]
  [    8     9    10]]]
Remapped image:
 [[[47662 35746 29789]
  [41704 23831     1]]

 [[11916 17873  5958]
  [53619 65535 59577]]]


In [33]:

remapped_image1 = remap_uint16_scrambled_contrast(annotation_np_uint16)
remapped_image2 = remap_uint16_scrambled_contrast(annotation_np_uint16)

In [36]:
imwrite('/nrs/spruston/Boaz/I2/annotation_rand1.tif', remapped_image1)

In [37]:
imwrite('/nrs/spruston/Boaz/I2/annotation_rand2.tif', remapped_image2)

In [38]:
np.unique(remapped_image2)

NDArrayITKBase([    1,    96,   191,   287,   382,   477,   573,   668,
                  764,   859,   954,  1050,  1145,  1241,  1336,  1431,
                 1527,  1622,  1718,  1813,  1908,  2004,  2099,  2195,
                 2290,  2385,  2481,  2576,  2671,  2767,  2862,  2958,
                 3053,  3148,  3244,  3339,  3435,  3530,  3625,  3721,
                 3816,  3912,  4007,  4102,  4198,  4293,  4389,  4484,
                 4579,  4675,  4770,  4865,  4961,  5056,  5152,  5247,
                 5342,  5438,  5533,  5629,  5724,  5819,  5915,  6010,
                 6106,  6201,  6296,  6392,  6487,  6583,  6678,  6773,
                 6869,  6964,  7059,  7155,  7250,  7346,  7441,  7536,
                 7632,  7727,  7823,  7918,  8013,  8109,  8204,  8300,
                 8395,  8490,  8586,  8681,  8777,  8872,  8967,  9063,
                 9158,  9253,  9349,  9444,  9540,  9635,  9730,  9826,
                 9921, 10017, 10112, 10207, 10303, 10398, 10494,

In [6]:
# Match H5 files by channels for all animals
logger.info(f"Matching H5 files in {base_dir}.")
animals_files = match_h5_files_by_channels(base_dir)

2024-11-14 12:50:16 - INFO - Matching H5 files in /nrs/spruston/Boaz/I2/2024-07-31_Mafe/.


In [7]:
animals_files

{'ANM542292_BM22': {'ch0': '/nrs/spruston/Boaz/I2/2024-07-31_Mafe/ANM542292_BM22/uni_tp-0_ch-0_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
  'ch1': '/nrs/spruston/Boaz/I2/2024-07-31_Mafe/ANM542292_BM22/uni_tp-0_ch-1_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
  'ch2': '/nrs/spruston/Boaz/I2/2024-07-31_Mafe/ANM542292_BM22/uni_tp-0_ch-2_st-0-x00-y00_obj-right_cam-long_etc.lux.h5'},
 'ANM543603_BM21': {'ch0': '/nrs/spruston/Boaz/I2/2024-07-31_Mafe/ANM543603_BM21/uni_tp-0_ch-0_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
  'ch1': '/nrs/spruston/Boaz/I2/2024-07-31_Mafe/ANM543603_BM21/uni_tp-0_ch-1_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
  'ch2': '/nrs/spruston/Boaz/I2/2024-07-31_Mafe/ANM543603_BM21/uni_tp-0_ch-2_st-0-x00-y00_obj-right_cam-long_etc.lux.h5'},
 'ANM543607_BM19': {'ch0': '/nrs/spruston/Boaz/I2/2024-07-31_Mafe/ANM543607_BM19/uni_tp-0_ch-0_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
  'ch1': '/nrs/spruston/Boaz/I2/2024-07-31_Mafe/ANM543607_BM19/uni_tp-0_ch-1_st-0-x00-y00_

In [8]:
# Get the files for the selected animal
files = animals_files.get(animal)
files

{'ch0': '/nrs/spruston/Boaz/I2/2024-07-31_Mafe/ANM543607_BM19/uni_tp-0_ch-0_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
 'ch1': '/nrs/spruston/Boaz/I2/2024-07-31_Mafe/ANM543607_BM19/uni_tp-0_ch-1_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
 'ch2': '/nrs/spruston/Boaz/I2/2024-07-31_Mafe/ANM543607_BM19/uni_tp-0_ch-2_st-0-x00-y00_obj-right_cam-long_etc.lux.h5'}

In [9]:
output_dir = os.path.join(base_dir, animal, 'itk')  # Ensure logs and outputs go to /itk
# Create the output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)
logger.info(f"Using output dir: {output_dir}")

2024-11-14 12:50:17 - INFO - Using output dir: /nrs/spruston/Boaz/I2/2024-07-31_Mafe/ANM543607_BM19/itk


In [ ]:
# Step 1: Register and Transform
logger.info("Starting registration and transformation.")
register_and_transform(fx, files, output_dir, param_files,logger , [2])
logger.info("Finished registration and transformation.")

In [ ]:
# Step 2: Compute Region Statistics
logger.info("Starting computation of region statistics.")
funcs = [np.mean, np.median, np.std]
num_cores = os.cpu_count()  # Automatically detect the number of cores
df_stats = compute_region_stats(files, output_dir, annotation_np, funcs, num_cores)
logging.info("Finished computation of region statistics.")

In [ ]:
# Log the location of the logs and output files
log_file = os.path.join(output_dir, f'registration_log_{animal}.txt')
logging.info(f"Processing logs and outputs are saved in: {log_file}")

# Multiple animals locally

In [1]:
animals = ['ANM549865_JFX673']

animal = 'ANM549865_JFX673'
base_dir = '/nrs/spruston/Boaz/I2/20240930_iDISCO_round2/'
fx_path = '/nrs/spruston/Boaz/I2/atlas10_hemi.tif'
param_files_dir = '/nrs/spruston/Boaz/I2/itk'
annotation_path = '/nrs/spruston/Boaz/I2/annotation_10_hemi.nii'

In [4]:
animals = ['ANM549057_left_JF522','ANM550749_left_JF552', 
           'ANM550751_left_JF673', 'ANM551089_left_JF673']
base_dir = '/nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains'
fx_path = '/nrs/spruston/Boaz/I2/atlas10_hemi.tif'
param_files_dir = '/nrs/spruston/Boaz/I2/itk'
annotation_path = '/nrs/spruston/Boaz/I2/annotation_10_hemi.nii'

In [5]:
# Load the annotation volume
logging.info(f"Loading annotation volume from {annotation_path}.")
itk_annotation = itk.imread(annotation_path, itk.ULL)
annotation_np = itk.array_view_from_image(itk_annotation)
annotation_np.dtype, annotation_np.shape


(dtype('uint64'), (800, 1320, 658))

In [6]:
# Match H5 files by channels for all animals
logging.info(f"Matching H5 files in {base_dir}.")
animals_files = match_h5_files_by_channels(base_dir)

In [7]:
for animal in animals:
    setup_logging(base_dir, animal)
    # Load the fixed image (fx)
    logging.info(f"Loading fixed image (fx) from {fx_path}.")
    fx = itk.imread(fx_path, pixel_type=itk.US)

    # Load the parameter files
    param_files = [
        os.path.join(param_files_dir, 'Order1_Par0000affine.txt'),
        os.path.join(param_files_dir, 'Order3_Par0000bspline.txt'),
        os.path.join(param_files_dir, 'Order4_Par0000bspline.txt'),
        os.path.join(param_files_dir, 'Order5_Par0000bspline.txt')
    ]
    logging.info(f"Loaded parameter files from {param_files_dir}.")
    # Get the files for the selected animal
    files = animals_files.get(animal)
    output_dir = os.path.join(base_dir, animal, 'itk')  # Ensure logs and outputs go to /itk
    # Create the output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    logging.info(f"Using output dir: {output_dir}")
    logging.info("Starting computation of region statistics.")
    funcs = [np.mean, np.median, np.std]
    num_cores = os.cpu_count()  # Automatically detect the number of cores
    df_stats = compute_region_stats(files, output_dir, annotation_np, funcs, num_cores)

INFO:ANM549057_left_JF522:Logging initialized for animal processing.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.


[<function mean at 0x14e76d0fb870>, <function median at 0x14e76d048ef0>, <function std at 0x14e76d0fba30>]


Processing labels: 100%|███████████████████████████████████████████████████████████| 687/687 [00:58<00:00, 11.83label/s]
INFO:ANM550749_left_JF552:Logging initialized for animal processing.


Region statistics saved to /nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains/ANM549057_left_JF522/itk/region_stats.csv


TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.


...

Region statistics saved to /nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains/ANM550751_left_JF673/itk/region_stats.csv


TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50838 (0xc696) encountered.
TIFFReadDirectory: Warning, Unknown field with tag 50839 (0xc697) encountered.


[<function mean at 0x14e76d0fb870>, <function median at 0x14e76d048ef0>, <function std at 0x14e76d0fba30>]


Processing labels: 100%|███████████████████████████████████████████████████████████| 687/687 [00:58<00:00, 11.74label/s]


Region statistics saved to /nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains/ANM551089_left_JF673/itk/region_stats.csv


In [9]:
paths = collect_region_stats_paths('/nrs/spruston/Boaz/I2/')

In [10]:
paths

{'555600': '/nrs/spruston/Boaz/I2/2024-09-09_Compare_iDISCO_EZCleat_THF/ANM555600_PSD-HT_iDISCO/itk/region_stats.csv',
 '549057': '/nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains/ANM549057_left_JF522/itk/region_stats.csv',
 '550749': '/nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains/ANM550749_left_JF552/itk/region_stats.csv',
 '550751': '/nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains/ANM550751_left_JF673/itk/region_stats.csv',
 '551089': '/nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains/ANM551089_left_JF673/itk/region_stats.csv'}

# Multiple animals using bsub

In [7]:
# Define parameters
# base_dir = '/nrs/spruston/Boaz/I2/2024-09-19_iDISCO_CalibrationBrains'
# base_dir = '/nrs/spruston/Boaz/I2/20240930_iDISCO_round2/'
# base_dir = '/nrs/spruston/Boaz/I2/20241023_iDisco_Run3_mousecity/'
# base_dir = '/nrs/spruston/Boaz/I2/20241104_iDISCO_R4/'
base_dir = '/nrs/spruston/Boaz/I2/2024-07-31_Mafe/'
fx = '/nrs/spruston/Boaz/I2/atlas10_hemi.tif'
param_files_dir = '/nrs/spruston/Boaz/I2/itk'
annotation_np = '/nrs/spruston/Boaz/I2/annotation_10_hemi.nii'
python_script = os.path.abspath('../src/main.py')
python_executable = sys.executable
num_cores = 64
flip_axes = []
flip_axes = 2 # for BM19 21, 22

In [8]:

animals_files = match_h5_files_by_channels(base_dir)
animals_files

{'ANM542292_BM22': {'ch0': '/nrs/spruston/Boaz/I2/2024-07-31_Mafe/ANM542292_BM22/uni_tp-0_ch-0_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
  'ch1': '/nrs/spruston/Boaz/I2/2024-07-31_Mafe/ANM542292_BM22/uni_tp-0_ch-1_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
  'ch2': '/nrs/spruston/Boaz/I2/2024-07-31_Mafe/ANM542292_BM22/uni_tp-0_ch-2_st-0-x00-y00_obj-right_cam-long_etc.lux.h5'},
 'ANM543603_BM21': {'ch0': '/nrs/spruston/Boaz/I2/2024-07-31_Mafe/ANM543603_BM21/uni_tp-0_ch-0_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
  'ch1': '/nrs/spruston/Boaz/I2/2024-07-31_Mafe/ANM543603_BM21/uni_tp-0_ch-1_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
  'ch2': '/nrs/spruston/Boaz/I2/2024-07-31_Mafe/ANM543603_BM21/uni_tp-0_ch-2_st-0-x00-y00_obj-right_cam-long_etc.lux.h5'},
 'ANM543607_BM19': {'ch0': '/nrs/spruston/Boaz/I2/2024-07-31_Mafe/ANM543607_BM19/uni_tp-0_ch-0_st-0-x00-y00_obj-right_cam-long_etc.lux.h5',
  'ch1': '/nrs/spruston/Boaz/I2/2024-07-31_Mafe/ANM543607_BM19/uni_tp-0_ch-1_st-0-x00-y00_

In [9]:
logs_dir = os.path.abspath('../logs')
src_dir = os.path.abspath('../')
# Create the logs directory if it doesn't exist
os.makedirs(logs_dir, exist_ok=True)
logs_dir

'/groups/spruston/home/moharb/DELTA_iDISCO/logs'

In [10]:

animals = ['ANM542292_BM22', 'ANM543603_BM21','ANM543607_BM19' ]


In [14]:
for animal in animals:
    # Construct the log file paths
    output_log = os.path.join(logs_dir, f'output_{animal}.log')
    error_log = os.path.join(logs_dir, f'error_{animal}.log')
    bsub_command = [
        'bsub',
        '-J', f'job_{animal}',  # Job name
        '-n', str(num_cores),  # Number of CPU cores
        '-o', output_log,  # Output log
        '-e', error_log,  # Error log
        'bash', '-c',  # Run the following as a bash command
        f"cd {src_dir} && {python_executable} {python_script} "
        f"--animal {animal} --base_dir {base_dir} --fx {fx} "
        f"--param_files_dir {param_files_dir} --annotation_np {annotation_np}"
        f" --flip_axes {flip_axes}"
    ]
    
    

    # Submit the job to the cluster
    print(f"Submitting job for animal: {animal}")
    print(bsub_command)
    subprocess.run(bsub_command)

Submitting job for animal: ANM542292_BM22
['bsub', '-J', 'job_ANM542292_BM22', '-n', '64', '-o', '/groups/spruston/home/moharb/DELTA_iDISCO/logs/output_ANM542292_BM22.log', '-e', '/groups/spruston/home/moharb/DELTA_iDISCO/logs/error_ANM542292_BM22.log', 'bash', '-c', 'cd /groups/spruston/home/moharb/DELTA_iDISCO && /groups/spruston/home/moharb/mambaforge/envs/pyants/bin/python /groups/spruston/home/moharb/DELTA_iDISCO/src/main.py --animal ANM542292_BM22 --base_dir /nrs/spruston/Boaz/I2/2024-07-31_Mafe/ --fx /nrs/spruston/Boaz/I2/atlas10_hemi.tif --param_files_dir /nrs/spruston/Boaz/I2/itk --annotation_np /nrs/spruston/Boaz/I2/annotation_10_hemi.nii --flip_axes 2']
Job <142016881> is submitted to default queue <local>.
Submitting job for animal: ANM543603_BM21
['bsub', '-J', 'job_ANM543603_BM21', '-n', '64', '-o', '/groups/spruston/home/moharb/DELTA_iDISCO/logs/output_ANM543603_BM21.log', '-e', '/groups/spruston/home/moharb/DELTA_iDISCO/logs/error_ANM543603_BM21.log', 'bash', '-c', 'cd 

This job will be billed to spruston
This job will be billed to spruston
This job will be billed to spruston


In [17]:
!bjobs

JOBID      USER    STAT  QUEUE      FROM_HOST   EXEC_HOST   JOB_NAME   SUBMIT_TIME
142016873  moharb  RUN   interactiv e02u30      3*h07u01    /bin/bash  Nov 14 12:49
142016881  moharb  RUN   local      h07u01      64*h06u07   *2292_BM22 Nov 14 12:52
142016882  moharb  RUN   local      h07u01      64*h06u09   *3603_BM21 Nov 14 12:52
142016883  moharb  RUN   local      h07u01      64*h06u10   *3607_BM19 Nov 14 12:52
